In [ ]:
import tensorflow as tf
import logictensornetworks as ltn
import pandas as pd
import numpy as np
import openpyxl
import commons

In [ ]:
#input the AntiHypertensiveTherapy dataset from the Vsac website
hpyertensive = pd.read_excel('../Haoming/AntiHypertensiveTherapy.xlsx', 'Expansion List')
hypertensive = hpyertensive.loc[12:, 'Unnamed: 0']
hypertensive = np.unique(hypertensive)
for i in range(len(hypertensive)):
    hypertensive[i] = 'rxn_' + hypertensive[i]
    hypertensive.append('rxn_' + hypertensive[i] + 'n')

In [ ]:
#input the target result (GOLD_multiclass.csv) we want our model to generate

target = pd.read_csv('../Haoming/target_result/GOLD_multiclass.csv', index_col=0)
target.drop(index=225, inplace=True) # no.225 data is lost

train_target = target[target['train'] == 1].drop(['train', 'test'], axis=1)
test_target = target[target['test'] == 1].drop(['train', 'test'], axis=1)
hypertension_target = train_target["Hypertension (int)"]
hypertension_test_target = test_target["Hypertension (int)"]

# absent = train_target[train_target == -1].index.tolist()
# questionable = train_target[train_target == 0].index.tolist()
# present = train_target[train_target == 1].index.tolist()
# unmentioned = train_target[train_target == 3].index.tolist()

In [ ]:
#input the feature matrix

feature_matrix = pd.read_csv('../Haoming/additional/_feature_matrix_all_sections_.csv', index_col=0)
feature_matrix.fillna(0, inplace=True)

In [ ]:
#seperate the fm to train feature and test feature also intersect with the AntiHypertensiveTherapy dataset

train_feature = feature_matrix.loc[train_target.index]
test_feature = feature_matrix.loc[test_target.index]
hypertension_feature = train_feature[train_feature.columns.intersection(hypertensive)]
hypertension_test_feature = test_feature[test_feature.columns.intersection(hypertensive)]
hypertensive = train_feature.columns.intersection(hypertensive)

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
#for negation you need to abstract the count in the big fm dataset and 
#add new column to count the negation count of each standard medical representation

In [ ]:
batch_size = 64 * strategey.num_replicas_in_sync
ds_train = tf.data.Dataset.from_tensor_slices((hypertension_feature,hypertension_target)).batch(batch_size)
ds_test = tf.data.Dataset.from_tensor_slices((hypertension_test_feature,hypertension_test_target)).batch(batch_size)
ds_train = strategy.experimental_distribute_dataset(ds_train)
ds_test = strategy.experimental_distribute_dataset(ds_test)

In [ ]:
size = len(hypertensive)
class MLP(tf.keras.Model):
    def __init__(self, n_classes, hidden_layer_sizes=(size,size,size/2)):
        super(MLP, self).__init__()
        self.denses = [tf.keras.layers.Dense(s, activation="elu") for s in hidden_layer_sizes]
        self.dense_class = tf.keras.layers.Dense(n_classes)
        self.dropout = tf.keras.layers.Dropout(0.2)
        
    def call(self, inputs, training=False):
        x = inputs
        for dense in self.denses:
            x = dense(x)
            x = self.dropout(x, training=training)
        return self.dense_class(x)

with strategy.scope():
    logits_model = MLP(3)
    p = ltn.Predicate(ltn.utils.LogitsToPredicateModel(logits_model,single_label=True))

In [ ]:
class_absent = ltn.Constant(0, trainable=False)
class_present = ltn.Constant(1, trainable=False)
class_questionable = ltn.Constant(2, trainable=False)

In [ ]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=2),semantics="forall")
Exists = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMean(p=6),semantics="exists")

formula_aggregator = ltn.Wrapper_Formula_Aggregator(ltn.fuzzy_ops.Aggreg_pMeanError(p=2))

In [ ]:
@tf.function
def axioms(feautures, labels, training=False):
    x = ltn.Variable("x", features)
    x_absent = ltn.Variable("x_absent", features[labels==-1])
    x_present = ltn.Variable("x_present", features[labels==1])
    x_questionable = ltn.Variable("x_questionable", features[labels==0])
    axioms = [
        Forall(x_absent, p([x_absent, class_absent], training=training)),
        Forall(x_present, p([x_present, class_present], training=training)),
        Forall(x_questionable, p([x_questionable, class_questionable], training=training)),
        Forall(x, Not(And(p([x, class_present]), p([x, class_absent])))),
        Forall(x, Not(And(p([x, class_present]), p([x, class_questionable])))),
        Forall(x, Not(And(p([x, class_questionable]), p([x, class_absent])))),
    ]
    

    return formula_aggregator(axioms).tensor

In [ ]:
for features, labels in ds_train:
    print("Initial sat level %.5f"%axioms(features,labels))
    break

In [ ]:
with strategy.scope():
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    metrics_dict = {
        'train_axiom_sat':tf.keras.metrics.Mean(name='train_axiom_sat'),
        'test_axiom_sat':tf.keras.metrics.Mean(name='test_axiom_sat')
        'train_accuracy': tf.keras.metircs.SparseCategoricalAccuracy(name="train_accuracy"),
        'test_accuracy': tf.keras.metircs.SparseCategoricalAccuracy(name="test_accuracy")
    }

In [ ]:
@tf.function
def train_step(features, labels):
    with tf.GradientTape() as tape:
        sat = axioms(features, labels, Training=True)
        loss_value = 1. - sat
    grads = tape.gradient(loss_value, p.trainable_variables)
    optimizer.apply_gradients(zip(grads, trainable_variables))
    sat = axioms(features, labels)
    metrics_dict['train_axiom_sat'](sat)
    predictions = logits_model(features)
    metrics_dict['train_accuracy'](tf.one_hot(labels, 3), predictions)
    

@tf.function
def test_step(features, labels):
    sat = axioms(features, labels)
    metrics_dict['test_axiom_sat'](sat)
    predictions = logits_model(features)
    metrics_dict['test_accuracy'](tf.one_hot(labels, 3), predictions)
    
@tf.function
def distributed_train_step(features, labels):
    strategy.run(train_step, args=(features, labels,))

@tf.function
def distributed_test_step(features, labels):
    strategy.run(test_step, args=(features, labels,))

In [ ]:
EPOCHS = 200

commons.train(
    EPOCHS,
    metrics_dict,
    ds_train,
    ds_test,
    distributed_train_step,
    distributed_test_step,
    csv_path='',
    track_metrics=20
)